In [1]:
%matplotlib inline

import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import lib.binvox_rw.binvox_rw as binvox_rw
import pickle
import PIL
import os

In [39]:
def resize_voxels(voxels, size):
    def resize_voxels_op(voxels, size):
        resized_voxels = voxels
        curr_shape = tf.shape(voxels)
        batch_size = curr_shape[0]
        curr_size = curr_shape[1]
        denom = curr_size // size

        for view, shape in [[[0, 3, 2, 1], [batch_size, curr_size, curr_size, size, denom]],
                            [[0, 3, 1, 2], [batch_size, size, curr_size, size, denom]], 
                            [[0, 1, 3, 2], [batch_size, size, size, size, denom]]]:
            resized_voxels = tf.reduce_any(tf.reshape(resized_voxels, shape), axis=4)
            resized_voxels = tf.transpose(resized_voxels, view)

        return resized_voxels

    tf.reset_default_graph()

    voxels_placeholder = tf.placeholder(bool, shape=voxels.shape)
    size_placeholder = tf.placeholder(tf.int32)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    print config
    sess.run(tf.global_variables_initializer())
    resized_voxels = sess.run(resize_voxels_op(voxels_placeholder, size_placeholder), 
                              feed_dict={voxels_placeholder: voxels, size_placeholder: size})

    return resized_voxels

def resize_images(image, size):
    return image.resize((size, size), PIL.Image.ANTIALIAS)

In [41]:
def load_voxels(model_path, model_ids, size=None):
    voxels = []
    
    for model_id in model_ids:
        binvox_id = "{}{}.binvox".format(model_path, model_id)
        
        with open(binvox_id, 'rb') as f:
            model = binvox_rw.read_as_3d_array(f)
            voxels.append(model.data)
            
    voxels = np.asarray(voxels, dtype=bool)
    
                 
                    
    if size is not None:
        voxels = resize_voxels(voxels, size)
        voxels = center_voxels(voxels)
    
    
    return voxels 

def center_voxels(voxels):
    
    
    sample_num, x_dim, y_dim, z_dim = voxels.shape
    min_x = x // 2
    min_y = y // 2
    min_z = z // 2
    max_x = x // 2
    max_y = y // 2
    max_z = z // 2
    
    step = 0
    for sample_i in voxels:
        
        #find min and max
        for xi in xrange(x):
            for yi in xrange(y): 
                for zi in xrange(z): 
                    if sample_i[xi, yi, zi] is True: 
                        if min_x > xi: 
                            min_x = xi
                        elif max_x < xi: 
                            max_x = xi

                        if min_y > yi: 
                            min_y = yi
                        elif max_y < yi: 
                            max_y = yi

                        if min_z > zi: 
                            min_z = zi
                        elif max_z < zi: 
                            max_z = zi

        x_mid = (max_x + min_x)//2 
        y_mid = (max_y + min_y)//2
        z_mid = (max_z + min_z)//2

        x_mid/=2
        y_mid/=2
        z_mid/=2
        
        
        # translate
        translate_z = z//2 - z_mid 
        translate_y = y//2 - y_mid
        translate_x = x//2 - x_mid
        zeros_like_sample_i = np.zeros_like(sample_i)
        zeros_like_sample_i[:, :, np.arange(0, z-translate_z) +translate_z] = sample_i[:, :, np.arange(0, z-translate_z)]
        zeros_like_sample_i[:, :, np.arange(0, y-translate_y) +translate_y] = sample_i[:, :, np.arange(0, y-translate_y)]
        zeros_like_sample_i[:, :, np.arange(0, x-translate_x) +translate_x] = sample_i[:, :, np.arange(0, x-translate_x)]
        voxels[i] = zeros_like_sample_i
        step+=1
        
    return voxels 

def load_ids(filename):
    df = pd.read_csv("data/metadata.csv")
    saved_column = df.fullId.values.astype(str)
    front = df.front.values(str)
    up = df.up.values(str)
    model_ids = np.char.lstrip(saved_column, "wss.")
    
    return model_ids, front, up

def load_images(image_path, model_ids, count,  size=None):
    images = [[] for _ in range(count)]
    step =0
    for model_id in model_ids:
        step+=1
        for i in range(count):
            image_file =  "{}{}/{}-{}.png".format(image_path, model_id, model_id, i)
            with PIL.Image.open(image_file) as image:
                resized = resize_images(image, size)
                images[i].append(np.asarray(resized))
            #if step%200 == 0: 
             #   print "Step ", step
           
    
    return np.asarray(images)

In [1]:
# creates the 6 faces from voxels
def voxel_to_faces(voxel):
    p0 = voxel.transpose((1,0,2)) # front y z x
    p1 = voxel.transpose((1,2,0)) # back y x z
    p2 = voxel.transpose((0,2,1)) # left z x y
    p3 = voxel                    # right z y x
    p4 = voxel.transpose((2,0,1)) # top x z y
    p5 = voxel.transpose((2,1,0)) # bottom x y z
    
    p = np.stack((p0, p1, p2, p3, p4, p5))
    faces = np.where(p.any(1), p.argmax(1), -1)

    return faces

# pass images in order front > back > left > right > top > bottom
# changes image orientation to match faces
def fix_image_orientation(images):
    fixed_images = np.stack([
        np.flip(images[0].transpose((1,0,2)), axis=1),
        np.flip(np.flip(images[1], axis=1), axis=0),
        np.flip(np.flip(images[2], axis=1), axis=0),
        np.flip(images[3].transpose((1,0,2)), axis=1),
        images[4].transpose((1,0,2)),
        np.flip(images[5].transpose((1,0,2)), axis=1)
    ])
             
    return fixed_images

# pass images in order front > back > left > right > top > bottom
# front refers to side facing us when plotted and not the front based on object
def color_voxel(voxel, images):
    colored_voxel = np.ones(voxel.shape + (3,))
    views = [
        (1,0,2,3),
        (0,2,1,3),
        (2,1,0,3),
        (0,2,1,3),
        (2,0,1,3),
        (0,2,1,3)
    ]
    
    faces = voxel_to_faces(voxel)
    orientated_images = fix_image_orientation(images)
    
    colors = np.zeros(faces.shape + (3,))
    colors[np.where(faces > -1)] = [0.5, 0.5, 0.5]
    
    for c, im in zip(colors, orientated_images):
        fig, axes = plt.subplots(1, 2, figsize=(10, 10))
        fig.subplots_adjust(hspace=0.1, wspace=0.1)
        
        ax = axes.flat[0]
        ax.imshow(c)
        ax.set_xlabel("Shape")

        # Plot the mixed-image.
        ax = axes.flat[1]
        ax.imshow(im[:,:,:3] / 255.0)
        ax.set_xlabel("Image")
        
        plt.show()

    for i, view in enumerate(views):
        face_mask = np.where(faces[i] > -1)
        colored_voxel = colored_voxel.transpose(view)
        colored_voxel[(faces[i][face_mask],) + face_mask] = orientated_images[i][face_mask][:,:3] / 255.
        
    print(np.sum(((voxel[(faces[3][np.where(faces[3] > -1)],) + np.where(faces[3] > -1)]) == False) * 1))
    
    return colored_voxel.transpose((2,1,0,3))

In [25]:
def save_data(voxels, views, path):
    os.makedirs(path)
    
    with open("{}voxels.pickle".format(path), 'wb') as f:
        pickle.dump(voxels, f)
        
    for i, view in enumerate(views):
        with open("{}view-{}.pickle".format(path, i), 'wb') as f:
            pickle.dump(view, f)
        
def read_data(path, num_views):
    views = []
    
    with open("{}voxels.pickle".format(path), 'rb') as f:
        voxels = pickle.load(f)
    
    for i in range(num_views):
        with open("{}view-{}.pickle".format(path, i), 'rb') as f:
            views.append(pickle.load(f))
    
    return voxels, views

In [26]:
model_path = "data/models-binvox/"
image_path = "data/screenshots/"
metadata_file = "data/metadata.csv"


model_ids, front, up = load_ids(metadata_file)
# Use above to load all model ids. Below is hardcoded due to missing files.
# model_ids = ['2fa5e7c9e378b2589cd62032ff4f127f', '3b30f1acc693d6a2308715fe727b3f72']



In [30]:
t = color_voxel(voxels[1], [images[4,1], images[5,1], images[1,1], images[0,1], images[3,1], images[2,1]])

(6802, 32, 32, 32)


In [27]:
model_path = "data/models-binvox/"
image_path = "data/screenshots/"
metadata_file = "data/metadata.csv"
voxels = load_voxels(model_path, model_ids, 32)


gpu_options {
  allow_growth: true
}



IOError: [Errno 24] Too many open files: 'data/screenshots/1004f30be305f33d28a1548e344f0e2e/1004f30be305f33d28a1548e344f0e2e-0.png'

In [42]:
images = load_images(image_path, model_ids, 6, 32)

Step  200
Step  200
Step  200
Step  200
Step  200
Step  200
Step  400
Step  400
Step  400
Step  400
Step  400
Step  400
Step  600
Step  600
Step  600
Step  600
Step  600
Step  600
Step  800
Step  800
Step  800
Step  800
Step  800
Step  800
Step  1000
Step  1000
Step  1000
Step  1000
Step  1000
Step  1000
Step  1200
Step  1200
Step  1200
Step  1200
Step  1200
Step  1200
Step  1400
Step  1400
Step  1400
Step  1400
Step  1400
Step  1400
Step  1600
Step  1600
Step  1600
Step  1600
Step  1600
Step  1600
Step  1800
Step  1800
Step  1800
Step  1800
Step  1800
Step  1800
Step  2000
Step  2000
Step  2000
Step  2000
Step  2000
Step  2000
Step  2200
Step  2200
Step  2200
Step  2200
Step  2200
Step  2200
Step  2400
Step  2400
Step  2400
Step  2400
Step  2400
Step  2400
Step  2600
Step  2600
Step  2600
Step  2600
Step  2600
Step  2600
Step  2800
Step  2800
Step  2800
Step  2800
Step  2800
Step  2800
Step  3000
Step  3000
Step  3000
Step  3000
Step  3000
Step  3000
Step  3200
Step  3200
Step  3200
S

In [ ]:
# calculate projection


In [43]:
save_path= "data/preprocessed/"


save_data(voxels, images, save_path)
voxels, images = read_data(save_path, 6)

In [22]:
print (model_ids.size)    

6802
